In [1]:
import numpy as np
from PIL import Image, ImageDraw
import time
import sys
import matplotlib.pyplot as plt
def Haar(scale = 1.2):  # scale in {1.2,2,2.8,3.6}
    ScaleToSize = {1.2:6, 2:8, 2.8:12, 3.6:16}
    HaarShape = ScaleToSize[scale]
    Haar_x = np.ones((HaarShape, HaarShape)) #[-1,1]
    Haar_y = np.ones((HaarShape, HaarShape)) # [1;-1]
    Haar_x[:, 0:int(HaarShape/2)] = -1
    Haar_y[int(HaarShape/2):HaarShape, :] = -1
    return (Haar_x, Haar_y, HaarShape)

def HCD_Kernel(img, scale = 1.2, NMS_Size = 7): # [1.2, 2, 2.8, 3.6]
    # HarrisDC, Get the Harris corner in img of scale
    (Haar_x, Haar_y, HaarShape) = Haar(scale)
    ScaleToNeighborSize = {1.2:7, 2:11, 2.8:15, 3.6:19}
    NeighborSize = ScaleToNeighborSize[scale]
    NMS_Side = int(NMS_Size/2)
    FinalSizeRed = HaarShape+NeighborSize+NMS_Size-3
    FinalShape = [img.shape[0]-FinalSizeRed, img.shape[1]-FinalSizeRed]

    # Step 0: Get final coordinate vector, vec_x, vec_y
    vec_x = np.zeros((FinalShape[0]*FinalShape[1],))
    vec_y = np.zeros((FinalShape[0]*FinalShape[1],))
    for idx in range(FinalShape[0]):
        for idy in range(FinalShape[1]):
            vec_x[idx*FinalShape[1]+idy] = idx
            vec_y[idx*FinalShape[1]+idy] = idy

    # step 1: perform convolution, get dx,dy,vec_dx,vec_dy
    dx = np.zeros((img.shape[0] - HaarShape + 1, img.shape[1] - HaarShape + 1))
    dy = np.zeros((img.shape[0] - HaarShape + 1, img.shape[1] - HaarShape + 1))
    for idx in range(HaarShape):
        for idy in range(HaarShape):
            dx = dx + img[idx:(idx + dx.shape[0]) , idy:(idy + dx.shape[1])] * Haar_x[idx, idy]
            dy = dy + img[idx:(idx + dx.shape[0]) , idy:(idy + dx.shape[1])] * Haar_y[idx, idy]
    Gap = int((NeighborSize+NMS_Size-2)/2)
    vec_dx = dx[Gap:-Gap,Gap:-Gap].flatten()
    vec_dy = dy[Gap:-Gap,Gap:-Gap].flatten()

    # Step 2: Construction C matrix, get det_trace,vec_det_trace
    c11 = np.zeros((dx.shape[0] - NeighborSize + 1, dx.shape[1] - NeighborSize + 1)) + 0.0001
    c12 = np.zeros((dx.shape[0] - NeighborSize + 1, dx.shape[1] - NeighborSize + 1)) + 0.0001
    c22 = np.zeros((dx.shape[0] - NeighborSize + 1, dx.shape[1] - NeighborSize + 1)) + 0.0001
    for idx in range(NeighborSize):
        for idy in range(NeighborSize):
            c11 = c11 + dx[idx:(idx + c11.shape[0]), idy:(idy + c11.shape[1])] ** 2
            c12 = c12 + dx[idx:(idx + c11.shape[0]), idy:(idy + c11.shape[1])] * \
                        dy[idx:(idx + c11.shape[0]), idy:(idy + c11.shape[1])]
            c22 = c22 + dy[idx:(idx + c11.shape[0]), idy:(idy + c11.shape[1])] ** 2   
    # Find out whether r = lambda_2/lambda_1 >= 0.1 and <=10
    # Or r/(1+r)^2 = det_c/trace_c^2 in [0.25,1]
    trace_c = c11 + c22               # lambda_1 + lambda_2
    det_c = c11*c22 - c12*c12         # lambda_1 * lambda_2
    det_trace = det_c/(trace_c**2)    
    Gap = int((NMS_Size-1)/2)
    vec_det_trace = det_trace[Gap:-Gap,Gap:-Gap].flatten()

    # Step 3: Non-maximum suppression over 3x3 region
    Center_Column = (NMS_Size*NMS_Size-1)/2
    Cpr = np.zeros((FinalShape[0] * FinalShape[1], NMS_Size*NMS_Size)) # A (H*W,9) matrix
    for idx in range(NMS_Size):
        for idy in range(NMS_Size):
            Cpr[:, idx*NMS_Size+idy] = det_trace[idx:(idx+FinalShape[0]), 
                                                 idy:(idy+FinalShape[1])].flatten()
    Cpr_rowmax = np.max(Cpr, axis = 1)
    Cpr_localmax = np.equal(Cpr_rowmax, Cpr[:, int(Center_Column)])
    return (vec_x, vec_y, np.abs(vec_dx), np.abs(vec_dy), vec_det_trace, Cpr_localmax, HaarShape)

In [2]:
def HCD_Feaure(vx, vy, vdx, vdy, vr, localmax, 
                           HaarShape,NeighborSize,NMS_Size,scale,
                           th_dxy = 100, th_r = 1/4.5, if_save = 1, input_image = "1"):
    WhetherCorner = np.logical_and(vdx>th_dxy, vr>=th_r); 
    WhetherCorner = np.logical_and(vdy>th_dxy, WhetherCorner);
    WhetherCorner = np.logical_and(localmax, WhetherCorner);  
    print("Scale = %f, Number of corners: %d" %(scale, np.sum(WhetherCorner)))

    bias = int((HaarShape+NeighborSize+NMS_Size-3)/2)
    if if_save:  # Save to image or not
        img_color = Image.open("HW4Pics/%s.jpg" %input_image)
        img_draw = ImageDraw.Draw(img_color)
        for idx,_ in enumerate(WhetherCorner):
            if(WhetherCorner[idx]==1):
                img_draw.ellipse((vy[idx]+bias-2, 
                                  vx[idx]+bias-2, 
                                  vy[idx]+bias+2, 
                                  vx[idx]+bias+2), 
                                 fill = 'red', outline ='red')
                #img_draw.point((vec_x[idx]+FinalSizeRed/2,vec_y[idx]+FinalSizeRed/2), fill =(255,0,0))
        del img_draw
        img_color.save("save_%s_%.1f.png" %(input_image,scale))

    FeatureVector = np.zeros((NeighborSize**2, np.sum(WhetherCorner))) # (Length, # Feature)
    Coord = np.zeros((2, np.sum(WhetherCorner))) # (2, # Feature) 
    HalfBias = int(NeighborSize/2)
    Cnt = 0
    for idx, value in enumerate(WhetherCorner):
        if(value==1):
            cx = int(vx[idx]+bias)
            cy = int(vy[idx]+bias)
            FeatureVector[:, Cnt] = img[cx-HalfBias:cx+HalfBias+1, cy-HalfBias:cy+HalfBias+1].flatten()
            Coord[0, Cnt] = cx
            Coord[1, Cnt] = cy
            Cnt = Cnt + 1
    return FeatureVector, Coord

In [3]:
def SSD(f1, f2):
    return np.sum((f1-f2)**2)
def NCC(f1, f2):
    m1 = np.mean(f1)
    m2 = np.mean(f2)
    top = np.sum((f1-m1)*(f2-m2))
    bottom = np.sqrt(np.sum((f1-m1)**2)*np.sum((f2-m2)**2))
    return top/bottom
def Correspondence(FM1, Coord1, FM2, Coord2, metric = "SSD"): #(L, #), (2, #), (L, ##), (2, ##)
    N1 = FM1.shape[1]
    N2 = FM2.shape[1]
    Pairs = []
    Scores = []
    if metric=="SSD":
        for idx1 in range(N1):
            Score = np.array([SSD(FM1[:,idx1], FM2[:,idx2]) for idx2 in range(N2)])
            idx2 = np.argmin(Score)
            Scores.append(np.min(Score))
            Pairs.append((idx1,idx2))
        # Select 100 pairs
        BestPairs = [x for _,x in sorted(zip(Scores,Pairs))]  # Incresing order
    if metric=="NCC":
        for idx1 in range(N1):
            Score = np.array([NCC(FM1[:,idx1], FM2[:,idx2]) for idx2 in range(N2)])
            idx2 = np.argmax(Score)
            Scores.append(np.max(Score))
            Pairs.append((idx1,idx2)) 
        # Select 100 pairs
        BestPairs = [x for _,x in sorted(zip(Scores,Pairs), reverse = True)]  # Decresing order   
    return BestPairs[0:int(np.min([100,len(BestPairs)]))]
def DrawPairs(input_image1, input_image2, Coord1, Coord2, Pairs, scale, metric = "SSD"):
    img_gray = Image.open("HW4Pics/%s.jpg" %input_image1)
    left = np.array(img_gray)
    img_gray = Image.open("HW4Pics/%s.jpg" %input_image2)
    right = np.array(img_gray)
    output = np.concatenate((left, right), axis = 1)
    img_color = Image.fromarray(output.astype("uint8"), mode = "RGB")
    img_draw = ImageDraw.Draw(img_color)
    for (idx1, idx2) in Pairs:
        x1 = Coord1[0,idx1]
        y1 = Coord1[1,idx1]
        x2 = Coord2[0,idx2]
        y2 = Coord2[1,idx2] + left.shape[1]
        img_draw.line([(y1,x1),(y2,x2)], fill =(255,0,0))
    del img_draw
    img_color.save("Coores_%s_%.1f_%s.png" %(input_image1, scale, metric))      

In [4]:
# Mapping in image 1
for scale in [1.2,2,2.8,3.6]:
    input_image1, scale0, NMS_Size, NeighborSize = ("1", 1.2, 7, 11)
    img_gray = Image.open("HW4Pics/%s.jpg" %input_image1).convert('L')
    img = np.array(img_gray).astype("float64")
    vx, vy, vdx, vdy, vr, localmax, HaarShape = HCD_Kernel(img, scale, NMS_Size)
    FeatureVector1, Coord1 = HCD_Feaure(vx, vy, vdx, vdy, vr, localmax, 
                               HaarShape,NeighborSize,NMS_Size,scale,
                               th_dxy = 100, th_r = 1/4.5, if_save = 1, input_image = input_image1)
    print("Saved output to save_%s_%.1f.png" % (input_image1,scale))

    input_image2, scale0, NMS_Size, NeighborSize = ("2", 1.2, 7, 11)
    img_gray = Image.open("HW4Pics/%s.jpg" %input_image2).convert('L')
    img = np.array(img_gray).astype("float64")
    vx, vy, vdx, vdy, vr, localmax, HaarShape = HCD_Kernel(img, scale, NMS_Size)
    FeatureVector2, Coord2 = HCD_Feaure(vx, vy, vdx, vdy, vr, localmax, 
                               HaarShape,NeighborSize,NMS_Size,scale,
                               th_dxy = 100, th_r = 1/4.5, if_save = 1, input_image = input_image2)
    print("Saved output to save_%s_%.1f.png" % (input_image2,scale))

    Pairs = Correspondence(FeatureVector1, Coord1, FeatureVector2, Coord2, metric = "NCC")
    DrawPairs(input_image1, input_image2, Coord1, Coord2, Pairs, scale, metric = "NCC")
    print("Find %s pairs" %len(Pairs))

Scale = 1.200000, Number of corners: 436
Saved output to save_1_1.2.png
Scale = 1.200000, Number of corners: 445
Saved output to save_2_1.2.png
Find 100 pairs
Scale = 2.000000, Number of corners: 574
Saved output to save_1_2.0.png
Scale = 2.000000, Number of corners: 519
Saved output to save_2_2.0.png
Find 100 pairs
Scale = 2.800000, Number of corners: 751
Saved output to save_1_2.8.png
Scale = 2.800000, Number of corners: 746
Saved output to save_2_2.8.png
Find 100 pairs
Scale = 3.600000, Number of corners: 644
Saved output to save_1_3.6.png
Scale = 3.600000, Number of corners: 645
Saved output to save_2_3.6.png
Find 100 pairs


In [5]:
# Mapping in image 2
for scale in [1.2,2,2.8,3.6]:
    input_image1, scale0, NMS_Size, NeighborSize = ("Truck1", 1.2, 7, 11)
    img_gray = Image.open("HW4Pics/%s.jpg" %input_image1).convert('L')
    img = np.array(img_gray).astype("float64")
    vx, vy, vdx, vdy, vr, localmax, HaarShape = HCD_Kernel(img, scale, NMS_Size)
    FeatureVector1, Coord1 = HCD_Feaure(vx, vy, vdx, vdy, vr, localmax, 
                               HaarShape,NeighborSize,NMS_Size,scale,
                               th_dxy = 100, th_r = 1/4.5, if_save = 1, input_image = input_image1)
    print("Saved output to save_%s_%.1f.png" % (input_image1,scale))

    input_image2, scale0, NMS_Size, NeighborSize = ("Truck2", 1.2, 7, 11)
    img_gray = Image.open("HW4Pics/%s.jpg" %input_image2).convert('L')
    img = np.array(img_gray).astype("float64")
    vx, vy, vdx, vdy, vr, localmax, HaarShape = HCD_Kernel(img, scale, NMS_Size)
    FeatureVector2, Coord2 = HCD_Feaure(vx, vy, vdx, vdy, vr, localmax, 
                               HaarShape,NeighborSize,NMS_Size,scale,
                               th_dxy = 100, th_r = 1/4.5, if_save = 1, input_image = input_image2)
    print("Saved output to save_%s_%.1f.png" % (input_image2,scale))
    
    Pairs = Correspondence(FeatureVector1, Coord1, FeatureVector2, Coord2, metric = "SSD")
    DrawPairs(input_image1, input_image2, Coord1, Coord2, Pairs, scale, metric = "SSD")
    print("Find %s pairs" %len(Pairs))

Scale = 1.200000, Number of corners: 500
Saved output to save_Truck1_1.2.png
Scale = 1.200000, Number of corners: 705
Saved output to save_Truck2_1.2.png
Find 100 pairs
Scale = 2.000000, Number of corners: 671
Saved output to save_Truck1_2.0.png
Scale = 2.000000, Number of corners: 851
Saved output to save_Truck2_2.0.png
Find 100 pairs
Scale = 2.800000, Number of corners: 795
Saved output to save_Truck1_2.8.png
Scale = 2.800000, Number of corners: 842
Saved output to save_Truck2_2.8.png
Find 100 pairs
Scale = 3.600000, Number of corners: 591
Saved output to save_Truck1_3.6.png
Scale = 3.600000, Number of corners: 696
Saved output to save_Truck2_3.6.png
Find 100 pairs


In [6]:
# Mapping in image 3
for scale in [1.2,2,2.8,3.6]:
    input_image1, scale0, NMS_Size, NeighborSize = ("Fountain1", 1.2, 7, 11)
    img_gray = Image.open("HW4Pics/%s.jpg" %input_image1).convert('L')
    img = np.array(img_gray).astype("float64")
    vx, vy, vdx, vdy, vr, localmax, HaarShape = HCD_Kernel(img, scale, NMS_Size)
    FeatureVector1, Coord1 = HCD_Feaure(vx, vy, vdx, vdy, vr, localmax, 
                               HaarShape,NeighborSize,NMS_Size,scale,
                               th_dxy = 100, th_r = 1/4.5, if_save = 1, input_image = input_image1)
    print("Saved output to save_%s_%.1f.png" % (input_image1,scale))

    input_image2, scale0, NMS_Size, NeighborSize = ("Fountain2", 1.2, 7, 11)
    img_gray = Image.open("HW4Pics/%s.jpg" %input_image2).convert('L')
    img = np.array(img_gray).astype("float64")
    vx, vy, vdx, vdy, vr, localmax, HaarShape = HCD_Kernel(img, scale, NMS_Size)
    FeatureVector2, Coord2 = HCD_Feaure(vx, vy, vdx, vdy, vr, localmax, 
                               HaarShape,NeighborSize,NMS_Size,scale,
                               th_dxy = 100, th_r = 1/4.5, if_save = 1, input_image = input_image2)
    print("Saved output to save_%s_%.1f.png" % (input_image2,scale))
    
    Pairs = Correspondence(FeatureVector1, Coord1, FeatureVector2, Coord2, metric = "NCC")
    DrawPairs(input_image1, input_image2, Coord1, Coord2, Pairs, scale, metric = "NCC")
    print("Find %s pairs" %len(Pairs))

Scale = 1.200000, Number of corners: 129
Saved output to save_Fountain1_1.2.png
Scale = 1.200000, Number of corners: 135
Saved output to save_Fountain2_1.2.png
Find 100 pairs
Scale = 2.000000, Number of corners: 208
Saved output to save_Fountain1_2.0.png
Scale = 2.000000, Number of corners: 213
Saved output to save_Fountain2_2.0.png
Find 100 pairs
Scale = 2.800000, Number of corners: 273
Saved output to save_Fountain1_2.8.png
Scale = 2.800000, Number of corners: 261
Saved output to save_Fountain2_2.8.png
Find 100 pairs
Scale = 3.600000, Number of corners: 198
Saved output to save_Fountain1_3.6.png
Scale = 3.600000, Number of corners: 205
Saved output to save_Fountain2_3.6.png
Find 100 pairs


In [7]:
# Mapping in image 4
for scale in [1.2,2,2.8,3.6]:
    input_image1, scale0, NMS_Size, NeighborSize = ("Tower1", 1.2, 7, 11)
    img_gray = Image.open("HW4Pics/%s.jpg" %input_image1).convert('L')
    img = np.array(img_gray).astype("float64")
    vx, vy, vdx, vdy, vr, localmax, HaarShape = HCD_Kernel(img, scale, NMS_Size)
    FeatureVector1, Coord1 = HCD_Feaure(vx, vy, vdx, vdy, vr, localmax, 
                               HaarShape,NeighborSize,NMS_Size,scale,
                               th_dxy = 100, th_r = 1/4.5, if_save = 1, input_image = input_image1)
    print("Saved output to save_%s_%.1f.png" % (input_image1,scale))

    input_image2, scale0, NMS_Size, NeighborSize = ("Tower2", 1.2, 7, 11)
    img_gray = Image.open("HW4Pics/%s.jpg" %input_image2).convert('L')
    img = np.array(img_gray).astype("float64")
    vx, vy, vdx, vdy, vr, localmax, HaarShape = HCD_Kernel(img, scale, NMS_Size)
    FeatureVector2, Coord2 = HCD_Feaure(vx, vy, vdx, vdy, vr, localmax, 
                               HaarShape,NeighborSize,NMS_Size,scale,
                               th_dxy = 100, th_r = 1/4.5, if_save = 1, input_image = input_image2)
    print("Saved output to save_%s_%.1f.png" % (input_image2,scale))
    
    Pairs = Correspondence(FeatureVector1, Coord1, FeatureVector2, Coord2, metric = "SSD")
    DrawPairs(input_image1, input_image2, Coord1, Coord2, Pairs, scale, metric = "SSD")
    print("Find %s pairs" %len(Pairs))

Scale = 1.200000, Number of corners: 437
Saved output to save_Tower1_1.2.png
Scale = 1.200000, Number of corners: 327
Saved output to save_Tower2_1.2.png
Find 100 pairs
Scale = 2.000000, Number of corners: 461
Saved output to save_Tower1_2.0.png
Scale = 2.000000, Number of corners: 344
Saved output to save_Tower2_2.0.png
Find 100 pairs
Scale = 2.800000, Number of corners: 405
Saved output to save_Tower1_2.8.png
Scale = 2.800000, Number of corners: 364
Saved output to save_Tower2_2.8.png
Find 100 pairs
Scale = 3.600000, Number of corners: 299
Saved output to save_Tower1_3.6.png
Scale = 3.600000, Number of corners: 257
Saved output to save_Tower2_3.6.png
Find 100 pairs
